In [ ]:
import numpy as np
import cmath
import pyperclip
import movement_assignment
import limbs
import universal_joint_angles

In [ ]:
class platform:
    def __init__(self,d,D,l1,l2,max_limb_error,max_limb_try_count,max_joint_angle):
        "d: distancia de cada junta universal en la plataforma al centro"
        
        "D: distancia del eje de cada motor al centro de la base"
        
        "la altura del eje del motor es conciderada el plano z=0"
        
        "l1: long del antebrazo"
        
        "l2: long del brazo"
        
        "max_limb_error: máximo error (en las mismas unidades de l1 y l2) que es permitido para posicionar el brazo"
        
        "max_limb_try_count: máxima cantidad de iteraciones para calcular el ángulo de un motor, si la cantidad"
        "Si la cantidad requerida de iteraciones es mayor, genera un error llamado 'Point unreachable' "
        
        "max_joint_angle:angulo máximo que puede tener la junta universal, una posición que lo exceda genera un error"
        "llamado 'Joint angle exceeded'"
        
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        
        self.d=d
        self.D=D

        self.l1=l1
        self.l2=l2
        
        self.max_limb_error=max_limb_error
        self.max_limb_try_count=max_limb_try_count
        
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        #defino los angulos máximos que se les permite rotar a los motores
        self.angle_max=90
        self.angle_min=-90
        #defino el ángulo máximo permitido en las juntas universales
        self.max_joint_angle=max_joint_angle
    def x_rotation_matrix(self,r_x):
        return np.array([[1,0,0],[0,np.cos(r_x),-np.sin(r_x)],[0,np.sin(r_x),np.cos(r_x)]]) 
    
    def y_rotation_matrix(self,r_y):
        return np.array([[np.cos(r_y),0,np.sin(r_y)],[0,1,0],[-np.sin(r_y),0,np.cos(r_y)]])
          
    def rotate(self,movement,joint,angle):
        if movement=="pitch":
            M_r=self.x_rotation_matrix(angle)
        if movement=="roll":
            M_r=self.y_rotation_matrix(angle)
            
        if joint=="C1":
            wm=np.identity(3)
        if joint=="C2":
            wm=self.get_m2_basis()
        if joint=="C3":
            wm=self.get_m3_basis()
        #transformamos los puntos a la base conveniente
        self.c1=np.linalg.solve(wm,self.c1)
        self.c2=np.linalg.solve(wm,self.c2)
        self.c3=np.linalg.solve(wm,self.c3)
        #realizamos la rotación de los mismos
        self.c1=M_r.dot(self.c1)
        self.c2=M_r.dot(self.c2)
        self.c3=M_r.dot(self.c3)
        #corregimos el offset de la plataforma
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #obtengo los puntos en la base original
        self.c1=wm.dot(self.c1)
        self.c2=wm.dot(self.c2)  
        self.c3=wm.dot(self.c3)
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
    def get_m2_basis(self):
        Wm2=np.array([[-np.cos(np.pi/3),np.cos(np.pi/6),0],[-np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm2

    def get_m3_basis(self):
        Wm3=np.array([[-np.cos(np.pi/3),-np.cos(np.pi/6),0],[np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm3
    
        
    def calculate_P(self):
            wm2=self.get_m2_basis()
            wm3=self.get_m3_basis()
            P1=self.c1+np.array([0,self.D,0])
            P2=np.linalg.solve(wm2,self.c2)+np.array([0,self.D,0])
            P3=np.linalg.solve(wm3,self.c3)+np.array([0,self.D,0])
            #elimino la coordenana en x
            P1=np.array([P1[1],P1[2]])
            P2=np.array([P2[1],P2[2]])
            P3=np.array([P3[1],P3[2]])
            return P1,P2,P3
    

        
    def correct_height(self,h):
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        offset=np.array([0,0,h-params[2]])
        #corrijo la altura
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        

    def solve_platform(self,r_x,r_y,h):
        if r_x==0 and r_y==0:
            #Este if se utiliza para no calcular rotaciones cuando los ángulos de ambos ejes son cero
            self.correct_height(h)
        else:
            joint,movement,angle=movement_assignment.assign_movement(r_x,r_y)
            self.rotate(movement,joint,angle)
            self.correct_height(h)
        
        P1,P2,P3=self.calculate_P()
        limb_i=limb(self.l1,self.l2)
        self.ang1=limb_i.calculate_motor_angle(P1,self.max_limb_error,self.max_limb_try_count)
        self.ang2=limb_i.calculate_motor_angle(P2,self.max_limb_error,self.max_limb_try_count)
        self.ang3=limb_i.calculate_motor_angle(P3,self.max_limb_error,self.max_limb_try_count)
        #Chekeo el ángulo necesario en las juntas universales
        return universal_joint_angles.check_angles(self)


    

        
    def generate_geogebra (self):
        s1=geogebra_script()
        #Imprimo el plato
        s1.add_point(self.c1,"C1")
        s1.add_point(self.c2,"C2")
        s1.add_point(self.c3,"C3")
        s1.define_polygon("C1","C2","C3")

        #imprimo los ejes de los motores
        D1=self.D*np.array([0,-1,0])
        D2=self.D*np.array([-1*np.sin(np.pi/3),np.cos(np.pi/3),0])
        D3=self.D*np.array([np.sin(np.pi/3),np.cos(np.pi/3),0])
        s1.add_point(D1,"M1")
        s1.add_point(D2,"M2")
        s1.add_point(D3,"M3")
        #Imprimo los planos de acción de los motores
        s1.add_point(np.array([0,0,0]),"AUX0") 
        s1.add_point(np.array([0,0,1]),"AUX4")
        s1.define_plane("M1","AUX0","AUX4")
        s1.define_plane("M2","AUX0","AUX4")
        s1.define_plane("M3","AUX0","AUX4")
        #imprimo el brazo l1
        a1=np.array([0,np.cos(self.ang1),np.sin(self.ang1)])*self.l1 #esto es visto desde el plano de referencia del motor
        a1=a1-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        s1.add_point(a1,"A1")
        s1.define_segment("M1","A1")
        s1.define_segment("A1","C1")

        #imprimo el brazo l2
        a2=np.array([0,np.cos(self.ang2),np.sin(self.ang2)])*self.l1  #esto es visto desde el plano de referencia del motor
        a2=a2-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        wm2=self.get_m2_basis()
        a2=wm2.dot(a2)
        s1.add_point(a2,"A2")
        s1.define_segment("M2","A2")
        s1.define_segment("A2","C2")
        #imprimo el brazo l3
        a3=np.array([0,np.cos(self.ang3),np.sin(self.ang3)])*self.l1  #esto es visto desde el plano de referencia del motor
        a3=a3-np.array([0,self.D,0])#muevo el cero al centro de la plataforma
        wm3=self.get_m3_basis()
        a3=wm3.dot(a3)
        s1.add_point(a3,"A3")
        s1.define_segment("M3","A3")
        s1.define_segment("A3","C3")        
        #imprimo los limites del motor
        aux1=np.array([0,-self.D-2,2.5])
        s1.add_point(aux1,"AUX1")
        aux2=np.array([5,-self.D-2,2.5])
        s1.add_point(aux2,"AUX2")
        s1.define_segment("AUX1","AUX2")
        return s1.print_script()
    
    def get_gradient(self):
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        return params
        

In [ ]:
class geogebra_script:
    def __init__(self):
        self.script=""
        self.first_item=1
        
    def add_point(self,point,name):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\"{0}=({1:.3f},{2:.3f},{3:.3f})\"\n".format(name,point[0],point[1],point[2])

    def add_2D_point(self,point,name):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\"{0}=({1:.3f},{2:.3f})\"\n".format(name,point[0],point[1])
        
    def define_polygon(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Polygon({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"

    def define_plane(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Plane({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"
    def define_segment(self,point1,point2):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Segment({},{})".format(point1,point2)
        self.script+="\""
        self.script+="\n"   
    def print_script(self):
        print("Execute[{\n"+self.script+"}]")
        return "Execute[{\n"+self.script+"}]"

In [ ]:
class limb:
    def __init__(self,l1,l2):
        self.l1=l1
        self.l2=l2
    def evaluate_angle(self,P,angle):
        a=self.l1*np.array([np.cos(angle),np.sin(angle)])
        b=((P-a)/(np.linalg.norm(P-a)))*self.l2
        return np.linalg.norm(P-a)-self.l2
     
    def calculate_motor_angle(self,P,max_error,tries):
        P_angle=cmath.polar(P[0]+1j*P[1])[1]
        angle_min=-90*np.pi/180
        angle_max=P_angle
        try_count=0
        while(try_count<tries):
            angle=0.5*(angle_max+angle_min)
            e=self.evaluate_angle(P,angle)
            if np.abs(e)<max_error:
                break
            if e>0:
                angle_min=angle
            if e<0:
                angle_max=angle
            try_count+=1
        if try_count==tries:
            #raise ValueError('Point unreachable\nP=({},{})'.format(P[0],P[1]))
            print('Point unreachable\nP=({},{})'.format(P[0],P[1]))
        else:
            return angle

In [ ]:
def check_motor_height (c1,c2,c3,D):
    h=2.5
    largo=2
    ancho=5
    
    c1x=c1[0]
    c2x=c2[0]
    c3x=c3[0]
        
    c1y=c1[1]
    c2y=c2[1]
    c3y=c3[1]
        
    c1z=c1[2]
    c2z=c2[2]
    c3z=c3[2]
        
    A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
    B=np.array([c1z,c2z,c3z]).transpose()
    params=np.linalg.solve(A,B)
    for x in np.linspace(start=0,stop=ancho,num=50):
        point=np.array([[x],[-D-largo],[1]])
        height_plato=np.dot(params,point)
        if height_plato<h:
            return -1


In [ ]:
angle_spacex=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
angle_spacey=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
h_space=np.linspace(start=7,stop=7,num=1)
s1=geogebra_script()
point=0
for height in [10,11,12,13,14]:
    for angle_x in angle_spacex:
        for angle_y in angle_spacey:
            p1=platform(d=6,D=10,l1=1,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
            result=p1.solve_platform(r_x=angle_x,r_y=angle_y,h=height) 
            if result==-1:
                print("angle exceeded")
                print("angle_x={}".format(angle_x*180/np.pi))
                print("angle_y={}".format(angle_y*180/np.pi))
            else:
                check_h=check_motor_height (p1.c1,p1.c2,p1.c3,p1.D)
                if check_h==-1:
                    print("Motor choca con el plato:\n")
                    "[BUG]:si el motor choca contra el plato y al mismo tiempo el punto es unreachable no se puede generar el"
                    "Script de geogebra, porque no hay angulos para armar los limbs"
                    #p1.generate_geogebra()
                P1,P2,P3=p1.calculate_P()
                name="P"+str(point)
                s1.add_2D_point(P1,name)
                point=point+1

In [ ]:
pyperclip.copy(s1.print_script())

In [49]:
p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
p1.solve_platform(r_x=-20*np.pi/180,r_y=-20*np.pi/180,h=10) 

[0.8660254 0.5       0.       ]
pitch
C3
-0.47536334624702803
Joint angle exceeded
 C1.angle=24.025911270594804º >23.0º


-1

In [50]:
pyperclip.copy(p1.generate_geogebra())

Execute[{
"C1=(0.000,-6.000,8.456)"
,"C2=(-5.196,3.000,8.456)"
,"C3=(4.332,2.501,12.575)"
,"Polygon(C1,C2, C3)"
,"M1=(0.000,-10.000,0.000)"
,"M2=(-8.660,5.000,0.000)"
,"M3=(8.660,5.000,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(M1,AUX0, AUX4)"
,"Plane(M2,AUX0, AUX4)"
,"Plane(M3,AUX0, AUX4)"
,"A1=(0.000,-5.136,-3.513)"
,"Segment(M1,A1)"
,"Segment(A1,C1)"
,"A2=(-4.448,2.568,-3.513)"
,"Segment(M2,A2)"
,"Segment(A2,C2)"
,"A3=(3.491,2.016,0.614)"
,"Segment(M3,A3)"
,"Segment(A3,C3)"
,"AUX1=(0.000,-12.000,2.500)"
,"AUX2=(5.000,-12.000,2.500)"
,"Segment(AUX1,AUX2)"
}]


In [46]:
p1.get_gradient()

array([ 0.36397023, -0.        , 10.        ])

In [40]:
joint,mov,angle=movement_assignment.assign_movement(-20*np.pi/180,-20*np.pi/180)

[1. 0. 0.]


In [28]:
joint

'C1'

In [29]:
mov

'roll'

In [30]:
angle*180/np.pi

20.0